In [ ]:
#  import main packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import pyarrow 

# define variables
file_version = ".1"
immo_vis_dir = "../../data/immo_vis/"
parquet_extension = ".parquet"
metropole_appartement_file = "ventes-metropole-appartement" + file_version  + parquet_extension
metropole_maison_file = "ventes-metropole-maison" +  file_version + parquet_extension
metropole_appartement_file_cleaned = metropole_appartement_file + "_step1_clean_" + parquet_extension

dpe_ranking     = ['0','A','B','C','D','E','F','F/G','G','NS','Unknown']
dpe_ranking_num = [0,1,2,3,4,5,6,7,7,0,0]

#  Set plot options
LargePlotActive = False
MediumPlotActive = False
SmallPlotActive = False
HeatMapDisplay = False

# chauffage système
chauffage_sys =["chauf_sys_climatisation","chauf_sys_radiateur","chauf_sys_pompe-chaleur","chauf_sys_chaudiere","chauf_sys_convecteur","chauf_sys_sol","chauf_sys_poele-bois"]
chauffage_mode =["chauffage_mode_individuel","chauffage_mode_collectif","chauffage_mode_central"]
chauffage_energie =["chauf_fioul","chauf_elec","chauf_gaz","chauf_bois"]
chauffage_systems = chauffage_sys + chauffage_mode + chauffage_energie

# expositions
expositions =['expo_has_nord', 'expo_has_sud', 'expo_has_est', 'expo_has_ouest']


# # functions

In [ ]:
# functions  

def print_numerical_isna(df) :
    columns = df.select_dtypes(include='number').columns.tolist()
    print (df[columns].isna().sum())

def print_categorial_isna(df) :
    columns = df.select_dtypes(include='category').columns.tolist()
    print (df[columns].isna().sum())     

def load_appartement_file () :
    start_path = Path(immo_vis_dir)
    final_path = start_path / metropole_appartement_file
    return pd.read_parquet(final_path.as_posix())

def save_appartement_file (df) :
    start_path = Path(immo_vis_dir)
    final_path = start_path / metropole_appartement_file_cleaned
    final_path.as_posix
    df.to_parquet(path=final_path.as_posix(),index=True)

def get_numerical_column_names (df) :
    return [ column for column  in df.columns if df[column].dtype not in ["object","category"]]


In [ ]:
# chauffage système
def display_heatmap(df, data_selection,target="prix_m2_vente",size=8,annotation=True) : 
    plt.figure(figsize=[size,size])

    data_selection.append(target)
    param = {
        'vmax': 1, 
        'vmin': -1, 
        'cmap': 'coolwarm', 
        'annot': annotation,
        'square': True,           # Cellules carrées
        'fmt': '.2f',            # Format des nombres (3 décimales)
        'cbar_kws': {'label': 'Corrélation'}  # Label de la colorbar
    }
    data = df.loc[:,data_selection].corr()
    display(data[target])
    sns.heatmap(data=data,**param)

# # decode file and display infos

In [ ]:
# import appartement file

start_path = Path(immo_vis_dir)
final_path = start_path / metropole_appartement_file
df = load_appartement_file()
nb_rows= df.shape[0]
nb_cols= df.shape[1]

In [ ]:
#  print main infos on appartmeent file


print ( f" {metropole_appartement_file} rows {nb_rows} columns {nb_cols}")
print (df.shape)
print (df.columns)
print (df.info())
print (df.describe())



In [ ]:
#  print columns modalities
for i in df.columns:
    print(f'modalities  {i} are : ',(df[i].nunique()))

In [ ]:
#  print comlmns nan values
for i in df.columns:
    print(f' {i} nan values {df[i].isna().sum()}  {(100*df[i].isna().sum()/len(df)):.2f}%')

#  process type_de_bien

In [ ]:
#  modify logement_neuf nan according to type de bien

df['logement_neuf'] = df['logement_neuf'].where(df['typedebien'] == "an", df['logement_neuf'].fillna('o'))
df['logement_neuf'] = df['logement_neuf'].where(df['typedebien'] == "a", df['logement_neuf'].fillna('n'))

print (df["logement_neuf"].isna().sum())

# replace an by a
df.loc[:,'typedebien'] ="a"

# process logement neuf as integer
df['logement_neuf'] = df['logement_neuf'].replace(['o','n'],[1,0])

print (df.typedebien.value_counts())
print (df.logement_neuf.value_counts())

print ("************** type de bien ***************************")
print ("keep only type de bien = a , logement neuf as integer 0 1")

## Drop columns that are not relevant

In [ ]:
#rendre index=idannonce pour retrouver la ligne initiale
df.set_index('idannonce',drop=True,inplace=True)

#supprimer les colonnes non pertinentes pour les maisons
df.drop(columns=['type_annonceur', 'typedetransaction','typedebien_lite',
       'prix_maison', 'prix_terrain', 'mensualiteFinance',"surface_terrain",
    #    'dpeC', #pas d'information supplémentaire sur dpeC par rapport à dpeL
    #    'date','eau',
       'categorie_annonceur', 'INSEE_COM', 'IRIS', 'duree_int',
       'TYP_IRIS_x', 'TYP_IRIS_y', 'GRD_QUART', 'UU2010', 
       'loyer_m2_median_n6', 'nb_log_n6', 'taux_rendement_n6',
       'loyer_m2_median_n7', 'nb_log_n7', 'taux_rendement_n7'], 
       inplace=True)

## Process prix_m2_vente

In [ ]:
# pre-analyse prix de vente au m2
if MediumPlotActive :
    plt.figure(figsize=(22,6))
    plt.subplot(151)
    sns.boxplot(df['prix_m2_vente'])
    plt.subplot(152)
    sns.histplot(df['prix_m2_vente'])
    plt.subplot(153)
    sns.scatterplot (df,x="surface",y="prix_m2_vente")
    plt.show()

df.describe()['prix_m2_vente']


In [ ]:


# remove outliers : 0,2%

colonnes_num_prim = ['surface',  'prix_m2_vente']

# Création du masque en utilisant apply
mask = df[colonnes_num_prim].apply(lambda x: (x >= x.quantile(0.002)) & (x <= x.quantile(0.998))| x.isna()).all(axis=1)
#ne garder que les lignes qui sont entre q=0.005 et q=0.995
df=df[mask]
#Vérification des résultats
df[colonnes_num_prim].describe()

(df['prix_m2_vente']*df['surface']-df['prix_bien']).describe()

if MediumPlotActive :
    plt.figure(figsize=(22,6))
    plt.subplot(151)
    sns.boxplot(df['prix_m2_vente'])
    plt.subplot(152)
    sns.histplot(df['prix_m2_vente'])
    plt.subplot(153)
    sns.scatterplot (df,x="surface",y="prix_m2_vente")
    plt.show()

# 
print ("************** prix_m2_vente : x >= x.quantile(0.002)) & (x <= x.quantile(0.998) ***************************")

## Process time series

In [ ]:
import datetime

df1 = df[["prix_m2_vente", "date"]]
print (df1.isna().sum())

print (df1.dtypes)
df1["date"] = pd.to_datetime(df1["date"],format="%Y-%m")
df1 = df1.set_index('date',drop=False)
print (df1.dtypes)
df1 = df1.sort_index(ascending=True)
df2 = df1.resample("MS").mean()
df2 = df2.interpolate()

display(df2.tail(12))

plt.figure(figsize=(16,6))
title = "prix vente m2 appartement over " + df2.index[0].strftime('%Y-%m') + " - " + df2.index[0-1].strftime('%Y-%m')
plt.title (title)
plt.ylabel('prix vente m2')
plt.xlabel('date')
plt.plot(df2.prix_m2_vente)

plt.axvline(datetime.datetime(2020, 1,9 ), color='red', linewidth=3, linestyle='-')
plt.text (datetime.datetime(2019, 6,1 ), 4000.,"start pandemic")

plt.axvline(datetime.datetime(2020, 3,17 ), color='red', linewidth=3, linestyle='--')
plt.axvline(datetime.datetime(2020, 5,11 ), color='red', linewidth=3, linestyle='--')

plt.axvline(datetime.datetime(2020, 10,30 ), color='red', linewidth=3, linestyle='--')
plt.axvline(datetime.datetime(2020, 12,15 ), color='red', linewidth=3, linestyle='--')

plt.axvline(datetime.datetime(2021, 4,3 ), color='red', linewidth=3, linestyle='--')
plt.axvline(datetime.datetime(2021, 5,3 ), color='red', linewidth=3, linestyle='--')

plt.axvline(datetime.datetime(2023, 1,1 ), color='green', linewidth=3, linestyle='-')
plt.text (datetime.datetime(2023, 1,1 ), 4000.,"end pandemic")

plt.show()
display (df2)

## recover inflation rate,  loan rate ...

In [ ]:

length = 18

start_path = Path(immo_vis_dir)
final_path = start_path / "inflation.csv"
inflation = pd.read_csv(final_path.as_posix())
inflation["date"] = pd.to_datetime(inflation["date"],format="%Y-%m")
display(inflation)

prix_scaled = (df2.prix_m2_vente -df2.prix_m2_vente.min()) / (df2.prix_m2_vente.max()-df2.prix_m2_vente.min())
prix_scaled = prix_scaled*inflation.inflation.max()
print (prix_scaled)


plt.figure(figsize=(16,6))
plt.plot (inflation.date, inflation.inflation,label="inflation rate")
plt.plot (inflation.date, inflation.taux_livret_A,label="taux livret A")
plt.plot (inflation.date, inflation.taux_BCE,label="taux BCE")
plt.plot (inflation.date, inflation.taux_pret_20ans,label="taux pret 20 ans")
plt.plot (inflation.date, prix_scaled,label="scaled prix vente m2")
plt.title ("inflation rate, investment rate and loan rate")
plt.ylabel("rate in %")
plt.legend()
plt.show()

from scipy.interpolate import  make_interp_spline
mean_price = df2.prix_m2_vente.mean()
xs =np.arange(len(df2.prix_m2_vente))
ys=df2.prix_m2_vente.to_numpy()

cs=make_interp_spline(xs,ys,k=3)

plt.figure(figsize=(16,4))
plt.plot (df2.date,df2.prix_m2_vente,label="prix vente m2")
# plt.plot (df2.date,mean_price+ cs(xs,1), label="smoothed prix vente m2")
plt.ylim((4000,5250))

plt.legend()








## Prophet Predictions

In [ ]:
from prophet import Prophet
length = 18

df3 = df2.rename(columns={"date" : "ds","prix_m2_vente":"y"})
df3 = pd.merge(df3,inflation,left_index=True,right_on="date")
df3.set_index(df2.index,inplace=True)
df3.drop("date",axis=1,inplace=True)

display(df3)
print (df3.columns)

df_train = df3[:-length]
df_test = df3[-length:]


model = Prophet (yearly_seasonality=True)

model.add_regressor('inflation')
model.add_regressor('taux_livret_A')
model.add_regressor('taux_BCE')
model.add_regressor('taux_pret_20ans')

attributes = ['ds', 'y', 'inflation','taux_livret_A','taux_BCE','taux_pret_20ans']
local_frame = df_train[attributes]
display(local_frame)
model.fit (local_frame)

future = model.make_future_dataframe(periods=length,freq="MS")
print("future")
display(future)
future = df_test[attributes].copy()
display(future)

forecast = model.predict(future)
display(forecast)

model.plot(forecast, ylabel = "prix vente m2",plot_cap=True)
plt.legend()
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(df_test['ds'],df_test["y"],label="test")
plt.plot(df_test['ds'],forecast["yhat"].iloc[-length:],label="forecast")
plt.ylabel('prix vente m2')
plt.xlabel('Months')
plt.xticks(df_test['ds'], rotation=45)

plt.fill_between(df_test['ds'], forecast['yhat_lower'].tail(length), forecast['yhat_upper'].tail(length), 
                 color='orange', alpha=0.15, label='Intervalle de confiance (80%)')
plt.legend()
plt.show()

# regressor_coefficients = {
#     'inflation': model.extra_regressors['inflation']['prior_scale'],
#     'taux livret A': model.extra_regressors['taux_livret_A']['prior_scale'],
#     'taux BCE': model.extra_regressors['taux_BCE']['prior_scale'],
#     'taux pret 20 ans': model.extra_regressors['taux_pret_20ans']['prior_scale']
# }
# plt.bar(regressor_coefficients.keys(), regressor_coefficients.values())
# plt.ylabel('Prior Scale')
# plt.title('Regressor Prior Scales')
# plt.tick_params(axis='x', rotation=45)
# plt.grid(True, alpha=0.3, axis='y')


,y,ds,inflation,taux_livret_A,taux_BCE,taux_pret_20ans
date,,,,,,
2019-02-01,4346.828981,2019-02-01 00:00:00.000000000,1.3,0.75,-0.4,1.5
2019-03-01,4396.564659,2019-03-01 00:00:00.000000000,1.1,0.75,-0.4,1.5
2019-04-01,4582.680229,2019-04-01 00:00:00.000000000,1.3,0.75,-0.4,1.5
2019-05-01,4643.971588,2019-05-01 00:00:00.000000000,0.9,0.75,-0.4,1.5
2019-06-01,4596.453575,2019-06-01 00:00:00.000000000,1.2,0.75,-0.4,1.5
...,...,...,...,...,...,...
2025-08-01,4452.751555,2025-08-01 00:00:00.000000000,0.9,1.70,3.0,3.2
2025-09-01,4691.698509,2025-09-01 00:00:00.000000000,1.2,1.70,3.0,3.2
2025-10-01,4537.504116,2025-10-01 00:00:00.000000000,0.9,1.70,3.0,3.2


Index(['y', 'ds', 'inflation', 'taux_livret_A', 'taux_BCE', 'taux_pret_20ans'], dtype='object')


,ds,y,inflation,taux_livret_A,taux_BCE,taux_pret_20ans
date,,,,,,
2019-02-01,2019-02-01,4346.828981,1.3,0.75,-0.40,1.5
2019-03-01,2019-03-01,4396.564659,1.1,0.75,-0.40,1.5
2019-04-01,2019-04-01,4582.680229,1.3,0.75,-0.40,1.5
2019-05-01,2019-05-01,4643.971588,0.9,0.75,-0.40,1.5
2019-06-01,2019-06-01,4596.453575,1.2,0.75,-0.40,1.5
...,...,...,...,...,...,...
2024-02-01,2024-02-01,4683.976148,3.0,3.00,4.00,3.8
2024-03-01,2024-03-01,4712.491391,2.3,3.00,4.00,3.8
2024-04-01,2024-04-01,4740.399683,2.2,3.00,3.75,3.8


17:17:48 - cmdstanpy - INFO - Chain [1] start processing
17:17:48 - cmdstanpy - INFO - Chain [1] done processing


future


,ds
0,2019-02-01
1,2019-03-01
2,2019-04-01
3,2019-05-01
4,2019-06-01
...,...
78,2025-08-01
79,2025-09-01
80,2025-10-01
81,2025-11-01


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().